# Lab | Summarization evaluation using LangSmith
Let's revisit your capstone project 2? Well, sort of. Pick diffierent sets of data and re-run this notebook. Maybe parts of the dataset you used in your last project week. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications using LangSmith.

What did you learn? - Let's discuss that in class

## LangSmith - LangChain evaluation

In [1]:
!pip install python-dotenv langchain openai huggingface_hub

from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
# Use either HUGGINGFACEHUB_API_TOKEN or HF_TOKEN - they're the same
HUGGINGFACEHUB_API_TOKEN = os.getenv("HF_TOKEN")


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="langsmith_max-test"

In [3]:
#Importing Client from Langsmith
from langsmith import Client
client = Client(api_key=LANGCHAIN_API_KEY)

### Create Dataset


In [4]:
from datasets import load_dataset
cnn_dataset = load_dataset(
    "ccdv/cnn_dailymail", 
    version="3.0.0",
    trust_remote_code=True
)

/Users/malibio/ironhack/lab-langsmith-summarizations/venv./lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def add_prefix(example):
    return {
        **example,
        "article": f"Summarize this news:\n{example['article']}"
    }

# Apply prefix to dataset
# cnn_dataset = cnn_dataset.map(add_prefix)

In [6]:
cnn_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [7]:
cnn_dataset['train'][0]

{'article': 'It\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It\'s a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but bec

In [8]:
# Get just a few news to test
MAX_NEWS = 10
sample_cnn = cnn_dataset["test"].select(range(MAX_NEWS)).map(add_prefix)

sample_cnn

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 10
})

The dataset contains three columns: article, highlights, and id. To use LangSmith, we need to create a dataset in LangSmith format.

LangSmith expects a prompt and a result. To achieve this, we will transform the article into a prompt by adding the prefix: "Summarize this news." As a result, we will use the content of highlights, which represents the summaries created by humans.

In [9]:
print(sample_cnn[0])

{'article': 'Summarize this news:\n(CNN)James Best, best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV\'s "The Dukes of Hazzard," died Monday after a brief illness. He was 88. Best died in hospice in Hickory, North Carolina, of complications from pneumonia, said Steve Latshaw, a longtime friend and Hollywood colleague. Although he\'d been a busy actor for decades in theater and in Hollywood, Best didn\'t become famous until 1979, when "The Dukes of Hazzard\'s" cornpone charms began beaming into millions of American homes almost every Friday night. For seven seasons, Best\'s Rosco P. Coltrane chased the moonshine-running Duke boys back and forth across the back roads of fictitious Hazzard County, Georgia, although his "hot pursuit" usually ended with him crashing his patrol car. Although Rosco was slow-witted and corrupt, Best gave him a childlike enthusiasm that got laughs and made him endearing. His character became known for his distinctive "kew-kew-kew" chuckle

Now We have the Dataset with the prompt and the Reference Summary, it is time to create a Dataset in LangSmith with this information.
### Create the Dataset in Langsmith

The dataset in LangSmith is composed of an input, which is the prompt passed to the model for evaluation, and an output, which should contain what we expect the model to return.

In [10]:
import datetime

In [11]:
import uuid

input_key = ['article']
output_key = ['highlights']

NAME_DATASET = f"Summarize_dataset_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

In [12]:
# Create dataset in LangSmith - Run only once to avoid POST errors
dataset = client.upload_dataframe(
    df=sample_cnn.to_pandas(),
    input_keys=input_key,
    output_keys=output_key,
    name=NAME_DATASET,
    description="Test Embedding distance between model summarizations",
    data_type="kv"
)

In this image, we can see an example from the dataset once it's been registered in LangSmith.

In the Input column, there is the prompt to be sent, while in the Output column, the expected output is stored.

When performing the comparison, the model will be given the prompt, and the Cosine distance between its response and the one stored in the sample dataset will be calculated.
<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_Dataset.jpg?raw=true">

### Recovering Models From Hugging Face
Let's retrieve both models from HuggingFace. A base T5 model and a model that has been fine-tuned using the training portion of this same dataset to generate summaries.

In [13]:
!pip install transformers torch

from transformers import pipeline
from langchain.llms.base import LLM
from typing import Optional, List, Any
import torch


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [14]:
class HuggingFaceSummarizer(LLM):
    pipeline: Any = None
    model_name: str = ""
    
    def __init__(self, model_name: str, **kwargs):
        super().__init__(model_name=model_name, **kwargs)
        
        # Use MPS if available (Apple Silicon), otherwise CPU
        device = 0 if torch.backends.mps.is_available() else -1
        
        self.pipeline = pipeline(
            "summarization", 
            model=model_name, 
            tokenizer=model_name,
            max_length=180,
            min_length=30,
            do_sample=False,
            device=device,
            torch_dtype=torch.float16 if torch.backends.mps.is_available() else torch.float32
        )
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        # Extract the actual article text after "Summarize this news:"
        if "Summarize this news:" in prompt:
            text = prompt.split("Summarize this news:")[-1].strip()
        else:
            text = prompt
        
        # Truncate if too long for the model
        if len(text) > 1000:
            text = text[:1000]
            
        result = self.pipeline(text)
        return result[0]['summary_text']
    
    @property
    def _llm_type(self) -> str:
        return "huggingface_summarizer"

summarizer_base = HuggingFaceSummarizer("t5-base")

Device set to use mps:0


In [15]:
summarizer_finetuned = HuggingFaceSummarizer("flax-community/t5-base-cnn-dm")

Device set to use mps:0


## Defining Evaluator
The first step is to define an evaluator, where we specify the variables we want to evaluate. In our case, I have chosen to measure only the "embedding_distance."

I've left the "string_distance" as a comment in case you want to conduct a test with two evaluations instead of one.


In [16]:
!pip install datasets transformers torch langchain-openai langsmith rapidfuzz==3.6.1

from langchain.smith import run_on_dataset, RunEvalConfig


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [17]:
# Using one of the multiple evaluators available on LangSmith
evaluation_config = RunEvalConfig(
    evaluators=[
        "embedding_distance",
        # "string_distance"  # Uncomment to add string distance evaluation
    ],
)

### Running Evaluator
With the same configuration, we can launch two evaluations on the same dataset. One for each of the chosen models.

In [18]:
project_name = f"T5-BASE_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

base_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=summarizer_base,
    evaluation=evaluation_config,
)

View the evaluation results for project 'T5-BASE_2025-06-24_18-24-41' at:
https://smith.langchain.com/o/d04f2d84-d849-4497-a494-ea7415629a4a/datasets/80257079-d062-4425-a703-a7e68fdd633e/compare?selectedSessions=5ddad187-8fbb-42d5-9010-03e4124e0476

View all tests for Dataset Summarize_dataset_2025-06-24_18-24-34 at:
https://smith.langchain.com/o/d04f2d84-d849-4497-a494-ea7415629a4a/datasets/80257079-d062-4425-a703-a7e68fdd633e
[------------------------------------------------->] 10/10

In [19]:
# Run evaluation for fine-tuned model
project_name = f"T5-FineTuned_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

finetuned_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=summarizer_finetuned,
    evaluation=evaluation_config,
)

View the evaluation results for project 'T5-FineTuned_2025-06-24_18-25-06' at:
https://smith.langchain.com/o/d04f2d84-d849-4497-a494-ea7415629a4a/datasets/80257079-d062-4425-a703-a7e68fdd633e/compare?selectedSessions=0c2725dc-86f9-4c01-8d80-13b1e6840ad7

View all tests for Dataset Summarize_dataset_2025-06-24_18-24-34 at:
https://smith.langchain.com/o/d04f2d84-d849-4497-a494-ea7415629a4a/datasets/80257079-d062-4425-a703-a7e68fdd633e
[------------------------------------------------->] 10/10

<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_Tests.jpg?raw=true">

In the image below you can see the comparision between two tests.
<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_CompareTestst.jpg?raw=true">

Well, since it has been so straightforward, why don't we try to make the comparison with an OpenAI model?

In [20]:
from langchain_openai import OpenAI
openai_llm = OpenAI(temperature=0.0)

In [21]:
project_name = f"OpenAI_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

openai_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=openai_llm,
    evaluation=evaluation_config,
)

View the evaluation results for project 'OpenAI_2025-06-24_18-25-30' at:
https://smith.langchain.com/o/d04f2d84-d849-4497-a494-ea7415629a4a/datasets/80257079-d062-4425-a703-a7e68fdd633e/compare?selectedSessions=782bc05f-074c-4a54-a85f-f7d8d187571e

View all tests for Dataset Summarize_dataset_2025-06-24_18-24-34 at:
https://smith.langchain.com/o/d04f2d84-d849-4497-a494-ea7415629a4a/datasets/80257079-d062-4425-a703-a7e68fdd633e
[------------------------------------------------->] 10/10

<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_CompareOpenAI_HF.jpg?raw=true">

The experiment with the OpenAI model has yielded the best results. But, be aware! As we can see, there is a cost involved since we are using an API, and it needs to be paid for.

Another crucial piece of information is that we can view performance data for the models. This data could also be useful for minimally evaluating our inference server.